This notebook compares the era5 model to the bouy data

In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import os
import math
import pickle
import matplotlib as mpl
import matplotlib.pyplot as plt
from tqdm import tqdm
from functools import reduce
import operator
from shapely import Point, LineString, Polygon, MultiPolygon
import cartopy
import time
#Enables the line profiler magic command #lprun
%load_ext line_profiler

In [3]:
#Extracts data from the dataset ds within the time_filter (tuple or timespan) interval for the 
#variable var_name found in the deph range deph_range in meters, positive is under water, negative above water
#It can be either a tuple (min,max) or a value it needs to equal
#Quality controll is made for position, deph, time, and the variable
#Note depth is the coordinate index while deph (without t) is the actual depth in meters 
#long_limits, lat_limits are limit tuples form the model. Thjey are used to filter geographically
#land_multipolygon is to filter land and close to shore data
def valid_data_extraction(ds, var_name, deph_range, time_filter, long_limits, lat_limits, land_multipolygon):
    if var_name not in ds.data_vars:
        raise ValueError(var_name, ' Not found')

    #Add longitude, latidude and position_qc as variables indexed by time,depth as all other variables
    TIME = ds['TIME'].values
    DEPTH = ds['DEPTH'].values
    n_DEPTHS = len(DEPTH)

    dataset_columns = {
        'LONG':ds['LONGITUDE'],
        'LAT':ds['LATITUDE'],
        'POS_QC':ds['POSITION_QC'],
    }

    ds_pos = xr.Dataset(
        data_vars=
        {k:(
            ["TIME", 'DEPTH'],
            np.repeat(np.reshape(v.values, (-1,1)), n_DEPTHS, axis=1),
            v.attrs,
        )for (k,v) in dataset_columns.items()},
        coords=dict(
            TIME=TIME,
            DEPTH=DEPTH,
        )
    ).drop_vars('DEPTH')
    ds = xr.merge([ds.drop_dims(['LATITUDE', 'LONGITUDE', 'POSITION']), ds_pos])
    
    #Filter for time of interest
    if type(time_filter) is tuple:
        ds = ds.sel(TIME=slice(time_filter[0], time_filter[1]))
    else:
        ds = ds.sel(TIME=time_filter)
    
    #Filter only avalible columns
    colum_names = [var_name]
    colum_names_qc = [var_name + '_QC']
    
    #Add fixed columns
    colum_names.extend(['LONG', 'LAT', 'DEPH'])
    colum_names_qc.extend(['DEPH_QC'])
    time_pos_qc = ['TIME_QC', 'POS_QC']
    
    #Filter for columns of interest
    ds = ds[colum_names + colum_names_qc + time_pos_qc]

    df = ds.to_dataframe()

    #Remove all rows with 0 bouy value
    #df = df[df[var_name] != 0]

    #Filter for data only within model limits
    geo_filter = (long_limits[0] <= df['LONG']) & (df['LONG'] <= long_limits[1]) & (lat_limits[0] <= df['LAT']) & (df['LAT'] <= lat_limits[1])
    df = df[geo_filter]

    #Filter data of df to only include data that does not lie close to shore
    #allow a small distance since points on the edge doeas not overlap according to shapely.overlaps 
    #df_shore_filter =  df.apply(lambda row: Point([row['LONG'], row['LAT']]).distance(land_multipolygon), axis=1) <= 0.00001
    #df = df[df_shore_filter]
    
    QC_good = [1.0, 7.0]
    #QC control for time and pos uses all of these values according to https://doi.org/10.13155/59938
    QC_time_pos_good = [1, 2, 5, 7, 8]
    
    #Filter the variable and depth for good quality data 
    filter_qc = [df[c_qc].isin(QC_good) for c_qc in colum_names_qc]
    #Filter for good time and pos 
    filter_qc.extend([df[c_qc].isin(QC_time_pos_good) for c_qc in time_pos_qc])
    #Add filter for deph value
    if type(deph_range) == tuple:
        filter_qc.append((deph_range[0] <= df['DEPH']) & (df['DEPH'] <= deph_range[1]))
    else:
        filter_qc.append(df['DEPH'] == deph_range)
    #Element-wise AND the filter
    filter_qc = reduce(operator.and_, filter_qc)
    df = df[filter_qc][colum_names]

    #Remove any duplicated measurements on different depths
    #To do so, first remove the depth from the index and then filter for unique time index by keeing the first
    df = df.reset_index('DEPTH')
    df = df[~df.index.duplicated(keep='first')]

    return df

In [4]:
#Searches the Appends
def search_file(file, data_dir, var_depth, model_result_columns, model_result_functions, time_filter, long_limits, lat_limits, land_multipolygon, model_ds, result_df):
    #Conditionally create the result dataframe
    if result_df is None:
        result_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in {
            'bouy_file_name':str,
            'bouy_longitude':float,
            'bouy_latitude':float,
            'bouy_time':np.dtype('<M8[ns]'), #np.datetime64
            'bouy_variable_name':str,
            'bouy_variable_value':float,
            'model_value':object,
            'model_longitude':float,
            'model_latitude':float,
            'model_time':np.dtype('<M8[ns]'),  #np.datetime64
        }.items()})

    #Load the data from the file
    file_path = os.path.join(data_dir, file)
    ds = xr.open_dataset(file_path)# , engine='scipy')
    ds_vars = list(ds.data_vars)

    #Filter for variables that exist in the data
    common_variables = set(var_depth.keys()).intersection(ds_vars)

    for var_name in common_variables:
        try:
            df = valid_data_extraction(ds, var_name, var_depth[var_name], time_filter[var_name], long_limits[var_name], lat_limits[var_name], land_multipolygon)
        except Exception as e:
            print(file, ' Could not extract data. Error: ' , e)
            continue
        
        if df.empty:
            continue
        else:
            #print(file, " : data found, shape: ", df.shape)
            pass

        #Drop unused columns and rename the other
        df = df.drop(labels=['DEPTH', 'DEPH'], axis=1).reset_index().rename(columns={
            'TIME': 'bouy_time',
            'LONG': 'bouy_longitude',
            'LAT':'bouy_latitude',
            var_name:'bouy_variable_value'})
        #Add bouy_file_name column
        df['bouy_file_name'] = file
        #Add bouy_variable_name column
        df['bouy_variable_name'] = var_name

        #Select the geographic region of interest, let time be the coordinate
        #Convert to dataframe reset indexing and rename the columns to signal model columns
        model_result_df = model_ds[var_name].sel(
            longitude=xr.DataArray(df['bouy_longitude'], dims="time"),
            latitude=xr.DataArray(df['bouy_latitude'], dims="time"),
            time=xr.DataArray(df['bouy_time'], dims="time"),
            method='nearest')[model_result_columns[var_name]].to_dataframe().reset_index().rename(columns={
                'time': 'model_time',
                'longitude': 'model_longitude',
                'latitude':'model_latitude'})
        
        #Creating the model_value column
        model_result_df['model_value'] = model_result_df.apply(model_result_functions[var_name], axis=1)
            
        #Concat model result with the bouy results
        df_concat = pd.concat([df, model_result_df], axis=1)

        #Filter nan values in the model columns
        df_concat = df_concat[df_concat[model_result_columns[var_name]].isna().apply(lambda x: not any(x), axis=1)]

        #Dropping the result columns
        df_concat = df_concat.drop(labels=model_result_columns[var_name], axis=1)

        result_df = pd.concat([result_df, df_concat])

    return result_df

In [5]:
#Loading significant wave height model 
model_swh_data_file = '/data/exjobb/sarssw/model/2021_swh_world_wide.nc'

model_swh_ds = xr.open_dataset(model_swh_data_file)
mod_swh_long_coord = model_swh_ds.coords['longitude'].values
model_swh_long_limits = (mod_swh_long_coord.min(), mod_swh_long_coord.max())

mod_swh_lat_coord = model_swh_ds.coords['latitude'].values
model_swh_lat_limits = (mod_swh_lat_coord.min(), mod_swh_lat_coord.max())

mod_swh_time_coord = model_swh_ds.coords['time'].values
model_swh_time_limits = (mod_swh_time_coord.min(), mod_swh_time_coord.max())

In [6]:
model_swh_ds

<xarray.Dataset>
Dimensions:    (longitude: 720, latitude: 361, time: 8760)
Coordinates:
  * longitude  (longitude) float32 0.0 0.5 1.0 1.5 ... 358.0 358.5 359.0 359.5
  * latitude   (latitude) float32 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * time       (time) datetime64[ns] 2021-01-01 ... 2021-12-31T23:00:00
Data variables:
    swh        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-02-21 12:23:29 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [7]:
#Loading wind speed model 
model_wspd_data_file = '/data/exjobb/sarssw/model/era5-wind-ww/all.nc'

model_wspd_ds = xr.open_dataset(model_wspd_data_file)
mod_wspd_long_coord = model_wspd_ds.coords['longitude'].values
model_wspd_long_limits = (mod_wspd_long_coord.min(), mod_wspd_long_coord.max())

mod_wspd_lat_coord = model_wspd_ds.coords['latitude'].values
model_wspd_lat_limits = (mod_wspd_lat_coord.min(), mod_wspd_lat_coord.max())

mod_wspd_time_coord = model_wspd_ds.coords['time'].values
model_wspd_time_limits = (mod_wspd_time_coord.min(), mod_wspd_time_coord.max())

In [8]:
model_wspd_ds

<xarray.Dataset>
Dimensions:    (latitude: 721, longitude: 1440, time: 8760)
Coordinates:
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * time       (time) datetime64[ns] 2021-01-01 ... 2021-12-31T23:00:00
Data variables:
    u10        (time, latitude, longitude) float32 ...
    v10        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:               CF-1.6
    history:                   Thu Feb 23 08:02:08 2023: ncrcat 2021_jan_feb_...
    NCO:                       4.7.2
    nco_openmp_thread_number:  1

In [9]:
#Load and create land multipolygon, buffered (expanded) to limit distance to shore
land_list = list(cartopy.feature.NaturalEarthFeature('physical', 'land', '50m').geometries())
polygon_list = []
for p  in land_list:
    if type(p) == MultiPolygon:
        polygon_list.extend(p.geoms)
    else:
        polygon_list.append(p)
land_multipolygon = MultiPolygon([p.buffer(0.01) for p in polygon_list])

In [10]:
bouy_data_dir = '/data/exjobb/sarssw/bouy/INSITU_GLO_PHYBGCWAV_DISCRETE_MYNRT_013_030/MO'

bouy_file_filter = [
    'GL_TS_MO_41121.nc', #Flips longitude sign in the middle of the data, from 66 to -66???! resutlts in asf search with over 7000 matches.
]

bouy_files = set(os.listdir(bouy_data_dir)).difference(bouy_file_filter)
#bouy_files = ['NO_TS_MO_6200146.nc'] #TODO remove once debugged
#bouy_files = ['NO_TS_MO_Butendiek.nc'] #TODO remove large amout of data, no overlap 
#bouy_files = ['GL_TS_MO_6100284.nc'] #TODO remove after filtering for 0 bouy values 


#Load data from survey and use only those files
#write_folder = '../bouy_survey/1h_survey'
#result_df_fn = 'result_df'
#with open(os.path.join(write_folder, result_df_fn),'rb') as f_r:
#    shore_survey_df = pickle.load(f_r)
#bouy_files = np.unique(shore_survey_df['bouy_file_name'])

result_df = None

var_depth = {
    'VHM0':0,
    'VAVH':0,
    'WSPD':(-30,0)
    }
model_result_columns = {
    'VHM0': ['swh'],
    'VAVH': ['swh'],
    'WSPD': ['u10', 'v10'],
    }

model_result_functions = {
    'VHM0': (lambda row: float(row['swh'])),
    'VAVH': (lambda row: float(row['swh'])),
    'WSPD': (lambda row: math.sqrt(row['u10']**2 + row['v10']**2)),
    }

model_time_limits = {
    'VHM0':model_swh_time_limits,
    'VAVH':model_swh_time_limits,
    'WSPD':model_wspd_time_limits,
}

model_long_limits = {
    'VHM0':model_swh_long_limits,
    'VAVH':model_swh_long_limits,
    'WSPD':model_wspd_long_limits,
}

model_lat_limits = {
    'VHM0':model_swh_lat_limits,
    'VAVH':model_swh_lat_limits,
    'WSPD':model_wspd_lat_limits,
}

models = {
    'VHM0':model_swh_ds,
    'VAVH':model_swh_ds,
    'WSPD':model_wspd_ds,
}
    
run_dict = {}
for bouy_file in tqdm(list(bouy_files)):
    #print(bouy_file)
    start = time.time()
    result_df = search_file(bouy_file, bouy_data_dir, var_depth, model_result_columns, model_result_functions, model_time_limits, model_long_limits, model_lat_limits, land_multipolygon, models, result_df)
    #%lprun -f search_file -f valid_data_extraction result_df = search_file(bouy_file, bouy_data_dir, var_depth, model_result_columns, model_result_functions, model_time_limits, model_long_limits, model_lat_limits, land_multipolygon, model_ds, result_df)
    end = time.time()
    run_dict[bouy_file] = end-start
    #print(end-start)


 26%|██▋       | 653/2487 [19:57<10:46,  2.84it/s]  

MO_TS_MO_NADR-S1.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')


 48%|████▊     | 1203/2487 [31:32<06:00,  3.56it/s]  

MO_TS_MO_Molo-Bandiera.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')


 56%|█████▋    | 1401/2487 [34:54<09:52,  1.83it/s]  

...

 75%|███████▌  | 1873/2487 [40:56<12:30,  1.22s/it]  

BS_TS_MO_BurgasBuoySURF.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')
BS_TS_MO_BurgasBuoySURF.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')


 93%|█████████▎| 2305/2487 [44:38<03:47,  1.25s/it]

BS_TS_MO_VarnaBuoySURF.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')
BS_TS_MO_VarnaBuoySURF.nc  Could not extract data. Error:  Timestamp('2021-01-01 00:00:00')


100%|██████████| 2487/2487 [46:53<00:00,  1.13s/it]


In [11]:
result_df

,bouy_file_name,bouy_longitude,bouy_latitude,bouy_time,bouy_variable_name,bouy_variable_value,model_value,model_longitude,model_latitude,model_time
0,GL_TS_MO_2100229.nc,131.110001,37.459999,2021-01-01 00:00:00,WSPD,10.999001,13.206324,131.0,37.5,2021-01-01 00:00:00
1,GL_TS_MO_2100229.nc,131.110001,37.459999,2021-01-01 01:00:00,WSPD,8.999000,12.732122,131.0,37.5,2021-01-01 01:00:00
2,GL_TS_MO_2100229.nc,131.110001,37.459999,2021-01-01 02:00:00,WSPD,6.999000,12.703803,131.0,37.5,2021-01-01 02:00:00
3,GL_TS_MO_2100229.nc,131.110001,37.459999,2021-01-01 03:00:00,WSPD,7.999000,12.518245,131.0,37.5,2021-01-01 03:00:00
4,GL_TS_MO_2100229.nc,131.110001,37.459999,2021-01-01 04:00:00,WSPD,6.999000,11.708543,131.0,37.5,2021-01-01 04:00:00
...,...,...,...,...,...,...,...,...,...,...
2075,GL_TS_MO_14040.nc,67.000000,-8.000000,2021-12-31 19:00:00,WSPD,1.579000,6.346973,67.0,-8.0,2021-12-31 19:00:00
2076,GL_TS_MO_14040.nc,67.000000,-8.000000,2021-12-31 20:00:00,WSPD,0.775000,6.676428,67.0,-8.0,2021-12-31 20:00:00
2077,GL_TS_MO_14040.nc,67.000000,-8.000000,2021-12-31 21:00:00,WSPD,0.474000,7.170267,67.0,-8.0,2021-12-31 21:00:00
2078,GL_TS_MO_14040.nc,67.000000,-8.000000,2021-12-31 22:00:00,WSPD,0.575000,7.488181,67.0,-8.0,2021-12-31 22:00:00


In [12]:
#Save resuld_df with pickle

write_folder = './model_bouy_comparison_no_zero_filter'
result_df_fn = 'result_df'

#Conditionally creates the folder for the result
os.makedirs(write_folder, exist_ok=True)

with open(os.path.join(write_folder, result_df_fn),'wb') as f_w:
    pickle.dump(result_df,f_w)


In [13]:
sorted(run_dict.items(), key=lambda x: x[1], reverse=True)

[('GL_TS_MO_23017.nc', 84.88056588172913),
 ('BS_TS_MO_URKA.nc', 78.33191108703613),
 ('GL_TS_MO_T0N165E.nc', 74.8471052646637),
 ('GL_TS_MO_T5N180W.nc', 71.5552544593811),
 ('NO_TS_MO_6200114.nc', 67.18783164024353),
 ('GL_TS_MO_T5S165E.nc', 55.75081753730774),
 ('GL_TS_MO_2200186.nc', 55.242018938064575),
 ('NO_TS_MO_ZeebruggeWeatherStation.nc', 54.920830726623535),
 ('GL_TS_MO_2200187.nc', 52.22821664810181),
 ('NO_TS_MO_6200146.nc', 51.55804467201233),
 ('GL_TS_MO_T8S165E.nc', 51.32682275772095),
 ('GL_TS_MO_2200102.nc', 49.63168144226074),
 ('BS_TS_MO_URSS.nc', 48.34000039100647),
 ('MO_TS_MO_ADN-MAMBO1.nc', 48.17768049240112),
 ('GL_TS_MO_2200107.nc', 46.272706747055054),
 ('GL_TS_MO_53006.nc', 44.07339334487915),
 ('NO_TS_MO_Gullfaks-C.nc', 43.585991621017456),
 ('NO_TS_MO_Oseberg-A.nc', 42.97952342033386),
 ('NO_TS_MO_6300110.nc', 42.6817672252655),
 ('GL_TS_MO_2200191.nc', 40.61321783065796),
 ('GL_TS_MO_2300492.nc', 40.58626866340637),
 ('NO_TS_MO_6200305.nc', 37.964701890945